In [120]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

In [121]:
data = pd.read_csv("../src/diabetes.csv")

In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
Number of times pregnant        768 non-null int64
Plasma glucose concentration    768 non-null int64
Diastolic blood pressure        768 non-null int64
Triceps skin fold thickness     768 non-null int64
2-Hour serum insulin            768 non-null int64
Body mass index                 768 non-null float64
Age                             768 non-null int64
Class                           768 non-null object
dtypes: float64(1), int64(6), object(1)
memory usage: 48.1+ KB


In [123]:
# split them into values
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [124]:
y = (y=='positive') + 0

In [125]:
# Standardization
sc = StandardScaler()
x = sc.fit_transform(x)

In [126]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train = sc.fit_transform(x_train)

In [127]:
class Network(nn.Module):
    def __init__(self, no_of_input_features, no_of_output_features):
        # We want to take all of the features of the Module class
        super(Network, self).__init__()
        # our attributes = Layers
        # our functionalities = Forward Propagation
        self.fc1 = nn.Linear(no_of_input_features, 50) # FC (Fully Connected Layer)
        self.fc2 = nn.Linear(50,50)
        self.fc3 = nn.Linear(50,no_of_output_features)
        # Define activation Function
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

In [128]:
net = Network(x_train.shape[1], 1)

In [129]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

In [130]:
loss_fn = nn.BCELoss(reduction='mean')

In [131]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [132]:
epochs = 600

In [133]:
for epoch in range(epochs):
    out = net(x_train)
    loss = loss_fn(out,y_train.reshape(-1,1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    accuracy = (((out.reshape(-1) > .5)+0)==y_train).sum().numpy()/x_train.shape[0]
    if (epoch+1) % 40 == 0:
        print("Epoch [{}/{}] and the accuracy is {:.3f} and the loss is {}".format(epoch+1, epochs, accuracy, loss))

Epoch [40/600] and the accuracy is 0.726 and the loss is 0.5400332808494568
Epoch [80/600] and the accuracy is 0.767 and the loss is 0.49708324670791626
Epoch [120/600] and the accuracy is 0.756 and the loss is 0.4891965389251709
Epoch [160/600] and the accuracy is 0.757 and the loss is 0.48699572682380676
Epoch [200/600] and the accuracy is 0.757 and the loss is 0.4854632019996643
Epoch [240/600] and the accuracy is 0.761 and the loss is 0.48401808738708496
Epoch [280/600] and the accuracy is 0.761 and the loss is 0.4825684130191803
Epoch [320/600] and the accuracy is 0.761 and the loss is 0.48108482360839844
Epoch [360/600] and the accuracy is 0.764 and the loss is 0.4795513451099396
Epoch [400/600] and the accuracy is 0.765 and the loss is 0.477956622838974
Epoch [440/600] and the accuracy is 0.765 and the loss is 0.47629404067993164
Epoch [480/600] and the accuracy is 0.769 and the loss is 0.47455987334251404
Epoch [520/600] and the accuracy is 0.767 and the loss is 0.4727548062801

In [134]:
x_test.shape

torch.Size([154, 7])

In [135]:
y_test.shape

torch.Size([154])

In [136]:
out = net(x_test)

In [137]:
(((out > .5)+0).reshape(-1).float() == y_test).sum().numpy()/x_test.shape[0]

0.8116883116883117